# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate


from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [3]:
# Create target data frame
df_area_burnt = fires_dt[['area']]
# Create the features data frame
fires_feat = fires_dt.drop(columns=['area'])

In [4]:
df_area_burnt.describe()

,area
count,517.000000
mean,12.847292
std,63.655818
min,0.000000
25%,0.000000
50%,0.520000
75%,6.570000
max,1090.840000


In [5]:
fires_feat.describe()

,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind,rain
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000


In [6]:
# Create training and test datasets
fires_feat_train, fires_feat_test, df_area_burnt_train, df_area_burnt_test = train_test_split(fires_feat, df_area_burnt, test_size=0.2, random_state=42)
#print(fires_feat_train.shape, fires_feat_test.shape, df_area_burnt_train.shape, df_area_burnt_test.shape)
# to shorten variable names
fires_feat = fires_feat_train
df_area_burnt = df_area_burnt_train
fires_dt = pd.concat([fires_feat,df_area_burnt], axis=1)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [7]:
# Numeric: scaled variables, no other transforms.

# Create dataframe of numberical features only
fires_feat_num = fires_feat._get_numeric_data()
fires_dt_num = fires_dt._get_numeric_data()
# Create a StandardScaler object
std_scaler = StandardScaler()
# Fit the StandardScaler object with the returns data
std_scaler.fit(fires_feat_num)
# std_scaler.mean_


StandardScaler()

In [8]:
# Transform the returns data using the fitted scaler
fires_feat_num_scaled = std_scaler.transform(fires_feat_num)
fires_feat_num_scaled = pd.DataFrame(fires_feat_num_scaled)
# Rename columns with previous column names
fires_feat_num_columns = fires_feat_num.columns
#type(fires_feat_num_columns) # pandas.core.indexes.base.Index
fires_feat_num_scaled.columns = fires_feat_num_columns
fires_feat_num_scaled.describe()

,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind,rain
count,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02,4.130000e+02
mean,4.301106e-18,-2.580664e-17,-2.222597e-15,-1.505387e-16,-1.881734e-17,-2.516147e-16,1.784959e-16,1.634420e-16,-8.172102e-17,-2.150553e-17
std,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00,1.001213e+00
min,-1.597691e+00,-1.923022e+00,-1.208091e+01,-1.689811e+00,-2.177122e+00,-2.253414e+00,-2.885369e+00,-1.783624e+00,-1.738767e+00,-6.781709e-02
25%,-7.248784e-01,-2.666965e-01,-5.607340e-02,-6.294833e-01,-4.504113e-01,-6.095469e-01,-5.856732e-01,-7.576675e-01,-7.282131e-01,-6.781709e-02
50%,-2.884720e-01,-2.666965e-01,1.793779e-01,-5.748427e-02,4.607517e-01,-9.583835e-02,3.680082e-02,-1.541638e-01,1.631241e-03,-6.781709e-02
75%,1.020747e+00,5.614662e-01,3.980112e-01,4.597489e-01,6.640359e-01,4.692410e-01,6.765658e-01,5.096903e-01,7.876175e-01,-6.781709e-02
max,1.893560e+00,3.874117e+00,9.530035e-01,2.724926e+00,1.248578e+00,3.577177e+00,2.492115e+00,3.346158e+00,3.033293e+00,1.984931e+01


In [9]:
# Categorical: one-hot encoding.

# Create dataframe of non-numberical features only
fires_feat_categ = fires_feat.drop(columns=fires_feat._get_numeric_data().columns)
#fires_feat_categ['month'].value_counts().plot(kind = 'bar')
#fires_feat_categ['day'].value_counts().plot(kind = 'bar')
# Create a OneHotEncoder object for month
onehot_m = OneHotEncoder(drop = 'first', handle_unknown='error') # handle_unknown='error' is by default
onehot_m.fit(fires_feat_categ[['month']])

OneHotEncoder(drop='first')

In [10]:
month_enc = onehot_m.transform(fires_feat_categ[['month']])
month_enc.toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [11]:
# Create a OneHotEncoder object for day
onehot_d = OneHotEncoder(drop = 'first', handle_unknown='error') # handle_unknown='error' is by default
onehot_d.fit(fires_feat_categ[['day']])

OneHotEncoder(drop='first')

In [12]:
day_enc = onehot_d.transform(fires_feat_categ[['day']])
day_enc.toarray()

array([[0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

After preprocessing of numerical features I got dataframe 

    - fires_feat_num_scaled

After preprocessing of non-numerical features I got two scipy objects: 

    - month_enc and 
    
    - day_enc

In [13]:
# Let's set objects for next section
X1_train_num = fires_feat_num_scaled.copy()
#print(X1_train_num.shape)
X1_train_cat = fires_feat_categ.copy()
#print(X1_train_cat.shape)
X1_train = pd.concat([X1_train_num.reset_index(drop=True),X1_train_cat.reset_index(drop=True)], axis=1)

Y1_train = df_area_burnt
#print(X1_train.columns)
#print(Y1_train.columns)

Let's create the transformer that will use the next section

In [14]:
# Build transformer
transformer1= ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'] ),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month', 'day']), 
    ], remainder='drop'
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [15]:
# Numeric: scaled variables, non-linear transformation to one or more variables.

# As per https://cwfis.cfs.nrcan.gc.ca/background/summary/fwi, "Canadian Forest Fire Weather Index (FWI) System consists of six components (...)"
# which are FFMC, DMC, DC, ISI and two (2) others. "Calculation of the components is based on consecutive daily observations of temperature, relative humidity, wind speed, and 24-hour precipitation."
# The four (4) features temp, RH, wind, rain contain information about the area burnt day only, similarly with ffmc and isi, while dmc and dc have been calculated usig wheather information from previous days.

In [16]:
# Let's plot the relationships to try to obtain inputs to decide what non-linear transformation to apply

def plot_feature_pairs(data, feature_names, color_labels=None, title_prefix=''):
    """
    Helper function to create scatter plots for all possible pairs of features.
    
    Parameters:
    - data: DataFrame containing the features to be plotted.
    - feature_names: List of feature names to be used in plotting.
    - color_labels: Optional. Cluster or class labels to color the scatter plots.
    - title_prefix: Optional. Prefix for plot titles to distinguish between different sets of plots.
    """
    # Create a figure for the scatter plots
    plt.figure(figsize=(60, 60))
    
    # Counter for subplot index
    plot_number = 1
    
    # Loop through each pair of features
    for i in range(len(feature_names)):
        for j in range(i + 1, len(feature_names)):
            plt.subplot(len(feature_names)-1, len(feature_names)-1, plot_number)
            
            # Scatter plot colored by labels if provided
            if color_labels is not None:
                plt.scatter(data[feature_names[i]], data[feature_names[j]], 
                            c=color_labels, cmap='viridis', alpha=0.7)
            else:
                plt.scatter(data[feature_names[i]], data[feature_names[j]], alpha=0.7)
            
            plt.xlabel(feature_names[i])
            plt.ylabel(feature_names[j])
            plt.title(f'{title_prefix}{feature_names[i]} vs {feature_names[j]}')
            
            # Increment the plot number
            plot_number += 1

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Get feature names
feature_names = fires_feat_num_scaled.columns

# Use the helper function to plot scatter plots without coloring by cluster labels
#plot_feature_pairs(fires_feat_num_scaled, feature_names, title_prefix='Original Data: ')

Observations:

- Several features show outliers, like rain, FFMC and ISI.

Let's remove some outliers. To remove some extreme outliers, we will use Interquartile Range (IQR), with outliers defined as values below Q1 - 2.5 * IQR or above Q3 + 2.5 * IQR.

But before that, rain is a very particular piece of information, that may appear to be more useful that it might be. "In the Canadian Forest Fire Weather Index (FWI) system, the "rain" component, or 24-hour precipitation, refers to the amount of rainfall recorded over the 24 hours preceding the noon observation time (usually 12:00 noon solar time) on the current day." And "In the Canadian Forest Fire Weather Index (FWI) system, the "burnt area" typically refers to the area affected by fire on a specific day, not the total area burned over the entire duration of a wildfire. If a fire lasts multiple days, the FWI system provides daily updates on fire intensity and potential spread, reflecting the conditions of each day." (Mix of actual federal link and Google Genesis)

If our dataset would contain actual date, it could be possible to observe the rain event in tomorrow outcome. For today's burnt area, there are few combinations in which the rain itself may have not been as relevant as appears.
If we open the .csv and order the dataset by rain, we observe that among the non-rain days, the area burnt =0 or =!0 is approx 50%-50%, and that for the rainy days it is 25%-75%, which is relevant. But actually the days that rained harder, there was area burnt, while the days with soft rain there was no area burnt at all. It is relevant if ir rained or not, it is not relevant how much, in this dataset. Likely, behind a rain day there were more rainy previous days than behind a non rainy day, pointing to when the terrain would have been more or less wet.

Due to the described above, because a predictive value is observed in its category and not in its numerical value, let's convert rain from numerical to categorical.

In [17]:
# drop rain column from numerical dataframe
fires_feat_num_scaled.drop(columns='rain', inplace = True)
fires_dt_num.drop(columns='rain', inplace = True)
# generate rain column in categorical dataframe and assign values
fires_feat_categ.loc[:, 'rain'] = fires_feat['rain'].apply(lambda rain: "dried" if rain == 0 else "rained")
print(fires_feat_categ.shape, fires_feat_categ.columns)

(413, 3) Index(['month', 'day', 'rain'], dtype='object')


The output variable area (burnt area) is very skewed towards zero (0.0). It could benefit from a log transformation, square root transformation or a Box-Cox transformation. Better to do it before removing outliers.

In [18]:
# Let's apply log transformation of variable area burnt

# Applying natural logarithm transformation
# Adding a small constant to avoid log(0)
fires_dt_num.loc[:,'log_area'] = np.log(fires_dt['area'] + 0.001)  # Using +1 ensures no zero issues
fires_dt_num.drop(columns='area', inplace = True)
print(fires_dt_num.shape, fires_dt_num.columns)
fires_dt_num.describe()

(413, 10) Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'log_area'],
      dtype='object')


,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind,log_area
count,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000
mean,4.661017,4.322034,90.533414,112.178692,549.813075,8.773123,18.887167,44.554479,3.997094,-2.326215
std,2.294222,1.208957,5.953240,65.814101,249.214533,3.897980,5.790388,16.590003,1.783362,4.494329
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.900000,-6.907755
25%,3.000000,4.000000,90.200000,70.800000,437.700000,6.400000,15.500000,32.000000,2.700000,-6.907755
50%,4.000000,4.000000,91.600000,108.400000,664.500000,8.400000,19.100000,42.000000,4.000000,-0.614336
75%,7.000000,5.000000,92.900000,142.400000,715.100000,10.600000,22.800000,53.000000,5.400000,1.882666
max,9.000000,9.000000,96.200000,291.300000,860.600000,22.700000,33.300000,100.000000,9.400000,6.615102


Remove outliers

In [19]:
# Now let's remove just few outliers. We need to check outliers on numeric features only but remove the rows from both numeric and categorical features

# First let's put numerical and categorical together
#fires_dt_num.columns, fires_feat_categ.columns
fires_num_cat = pd.concat([fires_dt_num, fires_feat_categ], axis=1)
print(fires_num_cat.shape, fires_num_cat.columns)


# Compute Q1 (25th percentile) and Q3 (75th percentile) for each column
print(fires_dt_num.shape)
Q1 = fires_dt_num.quantile(0.25)
Q3 = fires_dt_num.quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 2.5 * IQR
upper_bound = Q3 + 2.5 * IQR

# Find rows where any column has an outlier
outlier_rows = ((fires_dt_num < lower_bound) | (fires_dt_num > upper_bound)).any(axis=1)

# Remove rows that contain any outlier
fires_num_cat_logarea_noutlier = fires_num_cat[~outlier_rows]
fires_logarea_noutlier = fires_num_cat_logarea_noutlier[['log_area']]
fires_num_cat_noutlier = fires_num_cat_logarea_noutlier.drop(columns='log_area')
print(fires_num_cat_noutlier.shape, fires_num_cat_noutlier.columns)
print(type(fires_logarea_noutlier))
print(fires_logarea_noutlier.shape, fires_logarea_noutlier.columns)

(413, 13) Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'log_area', 'month', 'day', 'rain'],
      dtype='object')
(413, 10)
(389, 12) Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'month', 'day', 'rain'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
(389, 1) Index(['log_area'], dtype='object')


Now we need to separate numerical and categorical

In [20]:
# Create dataframe of numberical features only
fires_num_logarea_noutlier = fires_num_cat_logarea_noutlier._get_numeric_data()
p2_fires_feat_num_noutlier = fires_num_cat_noutlier._get_numeric_data()
#print(fires_num_logarea_noutlier.columns)
#print(p2_fires_feat_num_noutlier.columns)

# Create dataframe of categorical features only
fires_feat_categ = fires_num_cat_noutlier.drop(columns=fires_num_cat_noutlier._get_numeric_data().columns)
print(fires_feat_categ.shape, fires_feat_categ.columns)

(389, 3) Index(['month', 'day', 'rain'], dtype='object')


In [21]:
# Let's print it again
# Get feature names
feature_names = fires_num_logarea_noutlier.columns

# Use the helper function to plot scatter plots without coloring by cluster labels
#plot_feature_pairs(fires_num_logarea_noutlier, feature_names, title_prefix='Original Data: ')

The issue we were observing earlier in ffmc and isi is not observable anymore, discarding 413 - 389 = 24 rows, which is a 6% of our sample.

In [22]:
# Numeric: scaled variables

# Our dataframe of numberical features only, with log_area and with the outliers already removed is         p2_fires_feat_num_noutlier

# Create a StandardScaler object
std_scaler_p2 = StandardScaler()
# Fit the StandardScaler object with the returns data
std_scaler_p2.fit(p2_fires_feat_num_noutlier)
# std_scaler.mean_

StandardScaler()

In [23]:
# Transform the returns data using the fitted scaler
p2_fires_feat_num_noutlier_scaled = std_scaler_p2.transform(p2_fires_feat_num_noutlier)
p2_fires_feat_num_noutlier_scaled = pd.DataFrame(p2_fires_feat_num_noutlier_scaled)
# Rename columns with previous column names
p2_fires_feat_num_noutlier_columns = p2_fires_feat_num_noutlier.columns
#type(fires_feat_num_columns) # pandas.core.indexes.base.Index
p2_fires_feat_num_noutlier_scaled.columns = p2_fires_feat_num_noutlier_columns
p2_fires_feat_num_noutlier_scaled.describe()

,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind
count,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02
mean,1.506935e-16,5.251441e-17,3.717107e-15,1.872253e-16,-5.365602e-16,-1.872253e-16,3.653176e-16,1.598265e-16,8.676293e-17
std,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00,1.001288e+00
min,-1.580210e+00,-2.049661e+00,-2.948918e+00,-1.735582e+00,-2.259967e+00,-1.936663e+00,-3.001447e+00,-1.828457e+00,-1.727163e+00
25%,-6.982852e-01,-2.209497e-01,-3.548482e-01,-5.370505e-01,-3.958087e-01,-6.212115e-01,-5.532391e-01,-7.596348e-01,-7.230450e-01
50%,-2.573226e-01,-2.209497e-01,1.168008e-01,-7.000962e-02,4.370249e-01,-1.733982e-01,3.647528e-02,-1.309158e-01,2.151066e-03
75%,6.246026e-01,6.934059e-01,5.884499e-01,4.635304e-01,6.317938e-01,4.703335e-01,6.798000e-01,5.606751e-01,7.831315e-01
max,1.947490e+00,1.607761e+00,1.885485e+00,2.719864e+00,1.233468e+00,3.157214e+00,2.556164e+00,3.452782e+00,3.014504e+00


In [24]:
# Let's plot again. Now it is area log transformed, outliers removed, featrues only standardized, log_area added, all plotted
p2_fires_feat_num_noutlier_scaled['log_area'] = fires_num_logarea_noutlier['log_area']
# Get feature names
feature_names = p2_fires_feat_num_noutlier_scaled.columns

# Use the helper function to plot scatter plots without coloring by cluster labels
#plot_feature_pairs(p2_fires_feat_num_noutlier_scaled, feature_names, title_prefix='Original Data: ')

There is not a very noticeable difference before and after standardization in the plots between features. There is difference between log_area and every feature.

In [25]:
# Categorical: one-hot encoding.

# Recover dataframe of categorical features only
print(fires_feat_categ.columns, fires_feat_categ.shape)
#fires_feat_categ['month'].value_counts().plot(kind = 'bar')
#fires_feat_categ['day'].value_counts().plot(kind = 'bar')
# Create a OneHotEncoder object for month
onehot_m = OneHotEncoder(drop = 'first', handle_unknown='error') # handle_unknown='error' is by default
onehot_m.fit(fires_feat_categ[['month']])

Index(['month', 'day', 'rain'], dtype='object') (389, 3)


OneHotEncoder(drop='first')

In [26]:
month_enc = onehot_m.transform(fires_feat_categ[['month']])
month_enc.toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [27]:
# Create a OneHotEncoder object for day
onehot_d = OneHotEncoder(drop = 'first', handle_unknown='error') # handle_unknown='error' is by default
onehot_d.fit(fires_feat_categ[['day']])

OneHotEncoder(drop='first')

In [28]:
day_enc = onehot_d.transform(fires_feat_categ[['day']])
day_enc.toarray()

array([[0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [29]:
# Create a OneHotEncoder object for rain
onehot_r = OneHotEncoder(drop = 'first', handle_unknown='error') # handle_unknown='error' is by default
onehot_r.fit(fires_feat_categ[['rain']])

OneHotEncoder(drop='first')

In [30]:
rain_enc = onehot_r.transform(fires_feat_categ[['rain']])
rain_enc.toarray()

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [31]:
# Let's set objects for next section
p2_fires_feat_num_noutlier_scaled = p2_fires_feat_num_noutlier_scaled.drop(columns='log_area')
X2_train_num = p2_fires_feat_num_noutlier_scaled.copy()
print(X2_train_num.shape,X2_train_num.columns)
X2_train_cat = fires_feat_categ.copy()
print(X2_train_cat.shape,X2_train_cat.columns)
X2_train = pd.concat([X2_train_num.reset_index(drop=True),X2_train_cat.reset_index(drop=True)], axis=1)
print(X2_train.shape, X2_train.columns)

print(type(fires_logarea_noutlier))
Y2_train = fires_logarea_noutlier
print(Y2_train.shape, Y2_train.columns)

(389, 9) Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind'], dtype='object')
(389, 3) Index(['month', 'day', 'rain'], dtype='object')
(389, 12) Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'month', 'day', 'rain'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
(389, 1) Index(['log_area'], dtype='object')


Let's create the transformer that will use the next section

In [32]:
# Build transformer
transformer2 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind'] ),
        ('onehot', OneHotEncoder(drop = 'first', handle_unknown='error'), ['month', 'day', 'rain']), 
    ], remainder='drop'
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [33]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline(
    [
        ('preprocessing', transformer1), 
        ('regressor', LinearRegression())
    ]
)
pipe_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor', LinearRegression())])

In [34]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline(
    [
        ('preprocessing', transformer2), 
        ('regressor', LinearRegression())
    ]
)
pipe_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind']),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first'),
                                                  ['month', 'day', 'rain'])])),
                ('regressor', LinearRegression())])

In [35]:
# Pipeline C = preproc1 + advanced model


In [36]:
# Pipeline D = preproc2 + advanced model

    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

Pipeline A

In [37]:
#print(X1_train.shape)
#print(Y1_train.shape)
scoring = ['neg_root_mean_squared_error', 'r2']
pipe_A_dict = cross_validate(pipe_A, X1_train, Y1_train, cv=5, scoring = scoring, return_train_score = True)# , error_score='raise')
# In DataFrame form:
pd.DataFrame(pipe_A_dict)
pd.DataFrame(pipe_A_dict).mean()

fit_time                              0.010400
score_time                            0.007799
test_neg_root_mean_squared_error    -41.177356
train_neg_root_mean_squared_error   -43.439787
test_r2                              -0.315601
train_r2                              0.062168
dtype: float64

Pipeline B

In [38]:
#print(X2_train.shape)
#print(Y2_train.shape)
scoring = ['neg_root_mean_squared_error', 'r2']
pipe_B_dict = cross_validate(pipe_B, X2_train, Y2_train, cv=5, scoring = scoring, return_train_score = True)# , error_score='raise')
# In DataFrame form:
pd.DataFrame(pipe_B_dict)
pd.DataFrame(pipe_B_dict).mean()

c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
  File "c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "c:\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\pipeline.py", line 787, in predict
    Xt = transform.transform(Xt)
  File 

fit_time                             0.011200
score_time                           0.009800
test_neg_root_mean_squared_error    -4.520834
train_neg_root_mean_squared_error   -4.272089
test_r2                             -0.039276
train_r2                             0.097442
dtype: float64

Pipeline C

Pipeline D

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.